# Mapping Lichen Species in Edmonton

## Sample Code

## Updated on Feb 20, 2022

---

In [3]:
import arcpy

# access species feature classes in LichenProject folder using relative path
arcpy.env.workspace = r"..\LichenProject\LichenGDBs\LichenSpecies.gdb"
featureclasses = arcpy.ListFeatureClasses()

# check total # of spp existed and their names
print(len(featureclasses))   
print(featureclasses)

# reference to the current project, map, and layout
aprx = arcpy.mp.ArcGISProject("CURRENT")
mp = aprx.listMaps()[0]
lyt = aprx.listLayouts()[0]



In [4]:
import os

# loop to overlay a single spp feature class at each time, save it, then delete it
for fc in featureclasses:
    spp_source = os.path.join(r"..\LichenProject\LichenGDBs\LichenSpecies.gdb", fc)
    spp = arcpy.MakeFeatureLayer_management(spp_source, f"{fc}")
    
    # update the spp layer name to replace "_" with space
    spp_layer = mp.listLayers()[0]
    print(spp_layer.name)
    spp_layer.name= spp_layer.name.replace("_", " ")
    print(spp_layer)
    
    # create a legend, move the legend on the bottom left of the layout
    mf = lyt.listElements("MAPFRAME_ELEMENT", "Map Frame")[0]
    legend = lyt.listElements("LEGEND_ELEMENT")[0]
    legend.elementPositionX = mf.elementPositionX - 0.37
    legend.elementPositionY = mf.elementPositionY + 0.17
    
    # only include the legend for the spp name, remove all other layers
    for item in legend.items:
        if item.name != spp_layer:
            legend.removeItem(item)
            print( "deleted:" + item.name)
        elif item.name == spp_layer:
            print("legend contains: " + item.name)
        
    # export map as PNG and PAGX file 
    lyt.exportToPNG(r"..\LichenProject\LichenOutput\RawMap"+ fc + ".png", resolution = 300)
    
    # delete the currently added spp layer, then start the next spp layer
    mp.removeLayer(mp.listLayers()[0])   
    
print("finished")